
---

### This 📘 notebook needs to be run from VS Code with [these prerequisites](https://github.com/johnmaeda/SK-How-Tos#-prerequisites-to-getting-started-with-the-notebooks-in-this-repo).

How to use this notebook: 

* Just read the text and scroll along until you run into code blocks.
* Code blocks have computer code inside them — hover over the block and you can run the code.
* Run the code by hitting the ▶️ "play" button to the left. If the code runs you'll see a ✔️. If not, you'll get a ❌.
* The output and status of the code block will appear just below itself — you need to scroll down further to see it.
* Sometimes a code block will ask you for input in a hard-to-notice dialog box 👆 at the top of your notebook window. 

---

# 💬💨 Make your own super simple ChatGPT clone
The magic of chat using LLM AI is that it's extremely simple to implement. If you're still feeling unsure about semantic functions, catch up [here](https://learn.microsoft.com/en-us/semantic-kernel/howto/semanticfunctions). 

## Step 1️⃣: Let's get started by instantiating a kernel

You've already set up your API key information, so this should be an easy ▶️ and you're good to go.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, *-*"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.KernelExtensions;
using System.IO;
using Microsoft.SemanticKernel.Configuration;
using Microsoft.SemanticKernel.SemanticFunctions;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

// Grab the locally stored credentials from the settings.json file. Name the "backend" as "davinci" — assuming that you're using one of the davinci completion models. 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    kernel.Config.AddAzureOpenAICompletionBackend("davinci", model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAICompletionBackend("davinci", model, apiKey, orgId);

😱 **Get an error message?** The [first notebook](../s1e1-ez-starter-notebook/notebook.ipynb) walks you through this process so you should be all set. But if you're still stuck, go to https://aka.ms/sk/discord where we have realtime support.

## Step 2️⃣: Use the `ChatSkill.Chat` function to make a chatting AI in no time

We will be using the `Chat` semantic function that is accessible within the `ChatSkill` subdirectory of `skills`

```
📁 skills
│
└─── 📁 FunSkill
└─── 📁 ChatSkill
     |
     └─── 📁 Chat
     └─── 📁 ChatPersona
```

And with that, let's get the LLM AI to tell us talk to us.

### 💬 Create a simple chat loop with Semantic Kernel

You'll be surprised to see how easy it is to make a chat happen with semantic functions. It's easy as `history += <the new chat exchange>`!

In [ ]:
using Microsoft.SemanticKernel.Orchestration;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the Chat function from the FunSkill
var skill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "ChatSkill");

var myContext = new ContextVariables(); 
var botPrompt = "AI: Hello. What's your name?";
var history = $"{botPrompt}\n";

const int numberOfRounds = 4;

myContext.Set("history", history); 

for(var i = 0; i < numberOfRounds; i++) {
    try {
        // get input from the user and set the context variable
        Console.WriteLine("👆 Enter text in the input cell above to chat with the bot. 👆\n");
        var input = await InteractiveKernel.GetInputAsync(botPrompt);
        myContext.Set("input", input); 

        // run the chat function
        var myResult = await kernel.RunAsync(myContext,skill["Chat"]);

        // tack onto the history 👇 what's come back from the model
        /********************************************************/
        var theNewChatExchange = $"Me: {input}\nAI:{myResult}\n";
        history += theNewChatExchange;
        myContext.Set("history", history); 
        /********************************************************/
        // this way the new chat exchange gets passed into the next round

        // announce the number of rounds and the history
        Console.WriteLine($"Chat for {i+1} of {numberOfRounds} rounds with AI:\n{history}");

        // prepare to "prompt" the user with the bot's response
        botPrompt = $"AI: {myResult}";
    } catch {
        // if the user hits "Escape" we end the chat early
        Console.WriteLine("AI: Thanks for the wonderful chat!");
        break;
    }
}

### 🗯️ BONUS: Create a chat loop with a personality of your choice

The accrual of `history += theNewChatExchange` is a beautiful thing to notice. This is the basic mechanism whereby the chats you have with ChatGPT can feel so realistic. That's because the prompt is getting slightly longer and longer as you chat. The reason "it gets you" is because it hasn't forgotten what it's chatted with you in the past. 

Yet another facet to consider is how you're able to change the personality of the bot so easily. We'll use the `ChatSkill.ChatPersonality` function to do so with its added `$personality` context variable.

In [ ]:
using Microsoft.SemanticKernel.Orchestration;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the Chat function from the FunSkill
var skill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "ChatSkill");

var myContext = new ContextVariables(); 
var botName = "AI";

// Choose a personality here 👇 ...
/*********************************************************************/
var personality = "grumpy and extremely unhelpful";
/*********************************************************************/

var botPrompt = $"AI: Hello. My responses will be {personality}.";
var history = $"{botPrompt}\n";

const int numberOfRounds = 4;

myContext.Set("history", history); 
myContext.Set("personality", personality);

for(var i = 0; i < numberOfRounds; i++) {
    try {
        // get input from the user and set the context variable
        Console.WriteLine("👆 Enter text in the input cell above to chat with the bot. 👆\n");
        var input = await InteractiveKernel.GetInputAsync(botPrompt);
        myContext.Set("input", input); 

        // run the chat function
        var myResult = await kernel.RunAsync(myContext,skill["ChatPersonality"]);

        // tack onto the history what's come out
        var theNewChatExchange = $"Me: {input}\nAI:{myResult}\n";
        history += theNewChatExchange;
        myContext.Set("history", history); 

        // announce the number of rounds and the history
        Console.WriteLine($"Chat for {i+1} of {numberOfRounds} rounds with {botName}:\n{history}");

        // prepare to "prompt" the user with the bot's response
        botPrompt = $"AI: {myResult}";
    } catch {
        // if the user hits "Escape" we end the chat early
        Console.WriteLine($"AI: Thanks for the wonderful chat!");
        break;
    }
}

# ⏭️ Next Steps

Run through more advanced examples in the notebooks that are available in our GitHub repo at [https://aka.ms/sk/repo](https://aka.ms/sk/repo).

Or stay a longer while and change the `config.json` files to see how the different semantic functions behave. Or completely rewrite one of the functions that's been made available.